In [2]:
from flask import Flask, request, jsonify
import openai

In [3]:
# $5 will generate 15000 explanations

In [4]:
import os
try:
    with open("openai_token.txt", 'r') as file:
        token = file.readline().strip()
except FileNotFoundError:
    print("File not found:", file_path)

In [18]:
prompt_question_1 = "in a paragraph, explain the rationale behind the last move, where all previous moves are:"
prompt_question_2 = ", and last move is "

In [19]:
from openai import OpenAI
client = OpenAI(api_key = token)
setup  = prompt_question_1 + df.iloc[1_000]["prev_moves"] + prompt_question_2 + df.iloc[1_000]["last_move_fill"]
print(setup)
# temp = "e4 e5 Nf3 Nc6 Bc4 h6 O-O d6 d3 a6 a4 Bg4 Be3 Qf6 h3 Bxf3 gxf3 Be7 f4 exf4 Bd2 f3 Bc3"
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
#     model="gpt-4",
    messages=[{"role": "user", "content": setup}],
    stream=True,
)
print()
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

in a paragraph, explain the rationale behind the last move, where all previous moves are:d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3 Qxd3 O-O Bg5 Ng4, and last move is Bxe7

The last move, Bxe7, was played to exchange the bishop on e7 for the knight on d8. By doing this, White eliminates Black's knight, which was a strong defender of the king and a potential threat in future attacks. This move also opens up the f-file for White's rook, allowing for potential threats against Black's king. Additionally, by exchanging pieces, White simplifies the position and gains a slight material advantage, ultimately improving their position on the board.

In [6]:
import pandas as pd

In [14]:
df = pd.read_csv("prev_last_5000.csv")

In [15]:
# df = pd.DataFrame(df["progressive_moves"])
# df = pd.DataFrame(df["prev_moves"], df["last_move_fill"])
df.head()

,Unnamed: 0,prev_moves,last_move_fill
0,0,d4,d4
1,1,d4,d5
2,2,d4 d5,c4
3,3,d4 d5 c4,c6
4,4,d4 d5 c4 c6,cxd5


In [13]:
# df.iloc[12_000]["progressive_moves"]

'e4 e5 Nf3 Nc6 Bb5 a6 Bxc6 dxc6 Nxe5 Qd4 Nf3 Qxe4+ Qe2 Qxe2+ Kxe2 Nf6 d4 c5 c3 cxd4 cxd4 Bb4 Rd1 O-O a3 Re8+ Kf1 Bd6 Nc3 Ng4 h3 Nh2+ Nxh2 Bxh2 d5 Re5 g3 Rf5 Kg2 Rf6 Kxh2 Rxf2+ Kg1 Rf3 Rf1 Rxf1+ Kxf1 Bxh3+ Kg1 Rd8 Bf4 c6 d6 g5 Kh2 gxf4 Kxh3 fxg3 Rd1 Kg7 Kxg3 Kf6 Ne4+ Ke5 Ng5 Kf6 Nxh7+ Kg6 Rh1 Rh8 d7 Rg8 Kg4 Kg7 Ng5 Kf6 Rh6+ Ke5 Rh7 f6 Re7+ Kd6 Re8 Rxg5+ Kf4 Re5 d8=Q+ Kc5 Rxe5+ fxe5+ Kxe5 Kc4 Qb8 Kb3'

In [20]:
df.shape

(5000, 3)

In [21]:
df["explanation"] = ""
df.shape

(5000, 4)

In [23]:
df.tail()

,Unnamed: 0,prev_moves,last_move_fill,explanation
4995,4995,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2,Bxf3,
4996,4996,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2 Bxf3,gxf3,
4997,4997,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2 Bxf3 gxf3,cxd4,
4998,4998,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2 Bxf3 gxf...,exd4,
4999,4999,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2 Bxf3 gxf...,Nc6,


In [58]:
# df_prompt.at[0, "explanation"] = "checking"
# df_prompt

In [ ]:
for i in range(5000):
    stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", 
               "content": prompt_question_1 + df.iloc[i]["prev_moves"] + prompt_question_2 + df.iloc[i]["last_move_fill"]
              }],
    stream=True)
    
    explanation = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            explanation += chunk.choices[0].delta.content
            
    df.at[i, "explanation"] = explanation
    
    # every 100 will output
    if (i+1)/100 == 0:
        print(f"{i}. {df.iloc[i]['prev_moves']}--{df.iloc[i]['last_move_fill']}, explanation: {explanation}")

In [ ]:
# df_prompt.iloc[50]

In [ ]:
# df_prompt.to_csv("few_exp.csv")

In [ ]:
df.to_csv("div_move_exp_5000.csv")